In [598]:
import os 
import string 
import pandas as pd 
import numpy as np
from string import digits 
import matplotlib.pyplot as plt 

import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense 
from keras.models import Model 

# Get Dataset

In [599]:
train_data = pd.read_json('DATA/MT_JAVNRF_INZNTV/train_preprocess.json')
test_data = pd.read_json('DATA/MT_JAVNRF_INZNTV/test_preprocess.json')

In [600]:
train_data.drop(columns='id', inplace=True)
test_data.drop(columns='id', inplace=True)

In [601]:
train_data.head()

,text,label
0,"Wong kabèh padha gumun, temah padha takon-tin...","Mereka semua takjub, sehingga mereka memperbin..."
1,"Saka ing tutuké metu pedhang kang landhep, ka...",Dan dari mulut-Nya keluarlah sebilah pedang ta...
2,Déné kowé padha diawas! Aku wus ngandhakake...,Hati-hatilah kamu! Aku sudah terlebih dahulu m...
3,"supaya didhaftaraké bebarengan karo Maria, pa...","supaya didaftarkan bersama-sama dengan Maria, ..."
4,Allah sampun mungokaken Panjenenganipun saking...,Allah telah membangkitkan Dia dari antara oran...


In [602]:
test_data.head()

,text,label
0,"Nalika samana Hérodès, raja-wilayah, midhang...",Pada masa itu sampailah berita-berita tentang ...
1,"""Lan ing salebeting kawontenan ingkang makaten...","""Dan dalam keadaan demikian, ketika aku dengan..."
2,Margi saking punika panjenenganipun uninga sad...,Karena itu ia telah melihat ke depan dan telah...
3,Sih-rahmat nunggila ing kowé kabèh.,Kasih karunia menyertai kamu sekalian.
4,nganggo tanganku dhéwé aku nindakaké pagawe...,kami melakukan pekerjaan tangan yang berat. Ka...


# Data Preprocessing 

In [603]:
import nltk
nltk.download("punkt")
import re

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from string import punctuation

[nltk_data] Downloading package punkt to C:\Users\Kania Galih
[nltk_data]     Widowati\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [604]:
javanese_stopwords = pd.read_csv("local_languages_stopwords.csv")
javanese_stopwords.head()

,indonesian,javanese,sundanese
0,ada,Ana,Aya
1,adalah,yaiku,geus
2,adanya,orane,ayana
3,adapun,Kangge,Samentara éta
4,agak,Luwih,rada


In [605]:
#remove sundanese column
javanese_stopwords.drop(columns=["indonesian", "sundanese"], inplace=True)
javanese_stopwords.head()

,javanese
0,Ana
1,yaiku
2,orane
3,Kangge
4,Luwih


In [606]:
javanese_stopwords = javanese_stopwords['javanese'].tolist()
javanese_stopwords

['Ana',
 'yaiku',
 'orane',
 'Kangge',
 'Luwih',
 'Dianggo',
 'dadi ngono',
 'bakal',
 'bakal',
 'pungkasan',
 'pungkasan',
 'Pungkasane',
 'Aku',
 'kula',
 'banget',
 'Banget',
 'Sampeyan',
 'Sampeyan',
 'antarane',
 'antarane',
 'Antarane wong-wong mau',
 'Apa',
 'Apa',
 'Yen',
 'yaiku',
 'utamane',
 'Akatah',
 'Tegese',
 'Asal',
 'diwenehake',
 'ing',
 'utawa',
 'utawa iku',
 'utawa',
 'diwiwiti',
 'Wiwitane',
 'kaya',
 'kaya',
 'Kepiye',
 'Kepiye',
 'Nanging',
 'kanggo',
 'bagean',
 'Malah',
 'iku',
 'iku',
 'Apik',
 'bakal',
 'arep',
 'teka maneh',
 'Akeh',
 'Rama',
 'Anyar',
 'ngisor',
 'nomer saka',
 'Katon',
 'iki',
 'Kajaba iku',
 'Iki carane',
 'dadi',
 'Kajaba iku',
 'Iku iku',
 'Kajaba iku',
 'Nyambut gawe',
 'konco',
 'Mengko',
 'Durung',
 'durung',
 'Bener',
 'Iku bener',
 'Bener',
 'ana ing',
 'pungkasan',
 'pungkasan',
 'rampung',
 'Wonten pinten',
 'Apa',
 'Apa sing sampeyan lakoni',
 'apa wae',
 'Tegese',
 'Miwiti',
 'Macem-macem',
 'Tiba',
 'Menehi',
 'menehi',
 'Dip

In [607]:
nltk.download('stopwords', quiet=True)
indonesian_stopwords = stopwords.words('indonesian')

In [608]:
indonesian_stopwords

['ada',
 'adalah',
 'adanya',
 'adapun',
 'agak',
 'agaknya',
 'agar',
 'akan',
 'akankah',
 'akhir',
 'akhiri',
 'akhirnya',
 'aku',
 'akulah',
 'amat',
 'amatlah',
 'anda',
 'andalah',
 'antar',
 'antara',
 'antaranya',
 'apa',
 'apaan',
 'apabila',
 'apakah',
 'apalagi',
 'apatah',
 'artinya',
 'asal',
 'asalkan',
 'atas',
 'atau',
 'ataukah',
 'ataupun',
 'awal',
 'awalnya',
 'bagai',
 'bagaikan',
 'bagaimana',
 'bagaimanakah',
 'bagaimanapun',
 'bagi',
 'bagian',
 'bahkan',
 'bahwa',
 'bahwasanya',
 'baik',
 'bakal',
 'bakalan',
 'balik',
 'banyak',
 'bapak',
 'baru',
 'bawah',
 'beberapa',
 'begini',
 'beginian',
 'beginikah',
 'beginilah',
 'begitu',
 'begitukah',
 'begitulah',
 'begitupun',
 'bekerja',
 'belakang',
 'belakangan',
 'belum',
 'belumlah',
 'benar',
 'benarkah',
 'benarlah',
 'berada',
 'berakhir',
 'berakhirlah',
 'berakhirnya',
 'berapa',
 'berapakah',
 'berapalah',
 'berapapun',
 'berarti',
 'berawal',
 'berbagai',
 'berdatangan',
 'beri',
 'berikan',
 'berikut'

In [609]:
def preprocess_text(text, language_label):
    #lowercase
    text = text.lower()
    #remove quotes
    text = text.strip('"')
    # Combine removing special characters, punctuation, and extra whitespace
    text = re.sub(r"[^\w\s]", "", text)  # Removes special characters
    text = "".join([char for char in text if char not in punctuation])  # Removes punctuation
    text = " ".join(text.split())  # Removes extra whitespace
    #Tokenize 
    tokens = word_tokenize(text)

    # Select stopwords based on language label
    if language_label == "label":
        stopwords = indonesian_stopwords
    elif language_label == "text":
        stopwords = javanese_stopwords
    else:
        # Default to empty set if language label is not recognized
        stopwords = set()

    # Remove stopwords
    filtered_tokens = [word for word in tokens if word not in stopwords]
    

    # Join the tokens back into a string
    preprocessed_text = ' '.join(filtered_tokens)
    
    return preprocessed_text


In [610]:
train_data['label'] = train_data['label'].apply(lambda x: preprocess_text(x, indonesian_stopwords))
train_data['text'] = train_data['text'].apply(lambda x: preprocess_text(x, javanese_stopwords))

In [611]:
test_data['label'] = test_data['label'].apply(lambda x: preprocess_text(x, indonesian_stopwords))
test_data['text'] = test_data['text'].apply(lambda x: preprocess_text(x, javanese_stopwords))

In [612]:
train_data.head()

,text,label
0,wong kabeh padha gumun temah padha takontinako...,mereka semua takjub sehingga mereka memperbinc...
1,saka ing tutuke metu pedhang kang landhep kang...,dan dari mulutnya keluarlah sebilah pedang taj...
2,dene kowe padha diawas aku wus ngandhakake iki...,hatihatilah kamu aku sudah terlebih dahulu men...
3,supaya didhaftarake bebarengan karo maria paca...,supaya didaftarkan bersamasama dengan maria tu...
4,allah sampun mungokaken panjenenganipun saking...,allah telah membangkitkan dia dari antara oran...


In [613]:
train_data.isnull().sum()

text     0
label    0
dtype: int64

# Get Vocab 

In [614]:
# Add start and end tokens to target sequences
train_data['text'] = train_data['text'].apply(lambda x : 'START_ '+ x + ' _END')
test_data['text'] = test_data['text'].apply(lambda x : 'START_ '+ x + ' _END')

In [615]:
train_data.head()

,text,label
0,START_ wong kabeh padha gumun temah padha tako...,mereka semua takjub sehingga mereka memperbinc...
1,START_ saka ing tutuke metu pedhang kang landh...,dan dari mulutnya keluarlah sebilah pedang taj...
2,START_ dene kowe padha diawas aku wus ngandhak...,hatihatilah kamu aku sudah terlebih dahulu men...
3,START_ supaya didhaftarake bebarengan karo mar...,supaya didaftarkan bersamasama dengan maria tu...
4,START_ allah sampun mungokaken panjenenganipun...,allah telah membangkitkan dia dari antara oran...


In [616]:
test_data.head()

,text,label
0,START_ nalika samana herodes rajawilayah midha...,pada masa itu sampailah beritaberita tentang y...
1,START_ lan ing salebeting kawontenan ingkang m...,dan dalam keadaan demikian ketika aku dengan k...
2,START_ margi saking punika panjenenganipun uni...,karena itu ia telah melihat ke depan dan telah...
3,START_ sihrahmat nunggila ing kowe kabeh _END,kasih karunia menyertai kamu sekalian
4,START_ nganggo tanganku dhewe aku nindakake pa...,kami melakukan pekerjaan tangan yang berat kal...


In [617]:
all_ind_words=set()
for eng in train_data['label']:
    for word in eng.split():
        if word not in all_ind_words:
            all_ind_words.add(word)

all_javanese_words=set()
for jav in train_data['text']:
    for word in jav.split():
        if word not in all_javanese_words:
            all_javanese_words.add(word)

In [618]:
len(all_ind_words)

6758

In [619]:
len(all_javanese_words)

9575

In [620]:
train_data['length_jav_sentence']=train_data['text'].apply(lambda x:len(x.split(" ")))
train_data['length_ind_sentence']=train_data['label'].apply(lambda x:len(x.split(" ")))

In [621]:
test_data['length_jav_sentence']=test_data['text'].apply(lambda x:len(x.split(" ")))
test_data['length_ind_sentence']=test_data['label'].apply(lambda x:len(x.split(" ")))

In [622]:
train_data.head()

,text,label,length_jav_sentence,length_ind_sentence
0,START_ wong kabeh padha gumun temah padha tako...,mereka semua takjub sehingga mereka memperbinc...,31,24
1,START_ saka ing tutuke metu pedhang kang landh...,dan dari mulutnya keluarlah sebilah pedang taj...,36,34
2,START_ dene kowe padha diawas aku wus ngandhak...,hatihatilah kamu aku sudah terlebih dahulu men...,14,11
3,START_ supaya didhaftarake bebarengan karo mar...,supaya didaftarkan bersamasama dengan maria tu...,11,9
4,START_ allah sampun mungokaken panjenenganipun...,allah telah membangkitkan dia dari antara oran...,42,39


In [623]:
test_data.head()

,text,label,length_jav_sentence,length_ind_sentence
0,START_ nalika samana herodes rajawilayah midha...,pada masa itu sampailah beritaberita tentang y...,10,11
1,START_ lan ing salebeting kawontenan ingkang m...,dan dalam keadaan demikian ketika aku dengan k...,23,19
2,START_ margi saking punika panjenenganipun uni...,karena itu ia telah melihat ke depan dan telah...,34,31
3,START_ sihrahmat nunggila ing kowe kabeh _END,kasih karunia menyertai kamu sekalian,7,5
4,START_ nganggo tanganku dhewe aku nindakake pa...,kami melakukan pekerjaan tangan yang berat kal...,23,16


In [624]:
# Max length = 20
train_data=train_data[train_data['length_ind_sentence']<=20]
train_data=train_data[train_data['length_jav_sentence']<=20]

In [625]:
# Max length = 20
test_data=test_data[test_data['length_ind_sentence']<=20]
test_data=test_data[test_data['length_jav_sentence']<=20]

In [626]:
#Check Max Length 
print("maximum length Javanese Sentence ",max(train_data['length_jav_sentence']))
print("maximum length Indonesian Sentence ",max(train_data['length_ind_sentence']))

maximum length Javanese Sentence  20
maximum length Indonesian Sentence  20


In [627]:
#Check Max Length 
print("maximum length Javanese Sentence ",max(test_data['length_jav_sentence']))
print("maximum length Indonesian Sentence ",max(test_data['length_ind_sentence']))

maximum length Javanese Sentence  20
maximum length Indonesian Sentence  20


In [628]:
max_length_src=max(train_data['length_jav_sentence'])
max_length_tar=max(train_data['length_ind_sentence'])

In [629]:
max_length_src_test=max(test_data['length_jav_sentence'])
max_length_tar_test=max(test_data['length_ind_sentence'])

In [630]:
input_words = sorted(list(all_ind_words))
target_words = sorted(list(all_javanese_words))
num_encoder_tokens = len(all_ind_words)
num_decoder_tokens = len(all_javanese_words)
num_encoder_tokens, num_decoder_tokens

(6758, 9575)

In [631]:
num_decoder_tokens += 1 #for zero padding

In [632]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [633]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [634]:
train_data = shuffle(train_data)
train_data.head()

,text,label,length_jav_sentence,length_ind_sentence
1419,START_ yagene lenga nared iku ora diedol telun...,mengapa minyak narwastu ini tidak dijual tiga ...,17,15
1243,START_ dadi bab iku ora gumantung marang karep...,jadi hal itu tidak tergantung pada kehendak or...,18,16
2291,START_ banjur diwartani pangandikane pangeran ...,lalu mereka memberitakan firman tuhan kepadany...,12,14
2444,START_ nanging menawa aku nundhung setan ngang...,tetapi jika aku mengusir setan dengan kuasa al...,18,15
2809,START_ pangagemane jubah kang wis kacelup ing ...,dan ia memakai jubah yang telah dicelup dalam ...,14,14


In [635]:
test_data = shuffle(test_data)
test_data.head()

,text,label,length_jav_sentence,length_ind_sentence
46,START_ bareng petrus rawuh ing yerusalem wongw...,ketika petrus tiba di yerusalem orangorang dar...,14,14
449,START_ nanging dohana dongengdongeng gugontuho...,tetapi jauhilah takhayul dan dongeng neneknene...,11,10
877,START_ ingsun bakal enggal rawuh kukuhana apa ...,aku datang segera peganglah apa yang ada padam...,18,14
489,START_ yesus banjur ndangu maneh marang wongwo...,maka ia bertanya pula siapakah yang kamu cari ...,19,13
849,START_ amargi tresna dhateng bangsa kita saha ...,sebab ia mengasihi bangsa kita dan dialah yang...,16,13


In [636]:
# Combine the DataFrames
combined_data = pd.concat([train_data, test_data], ignore_index=True)

In [637]:
combined_data.head()

,text,label,length_jav_sentence,length_ind_sentence
0,START_ yagene lenga nared iku ora diedol telun...,mengapa minyak narwastu ini tidak dijual tiga ...,17,15
1,START_ dadi bab iku ora gumantung marang karep...,jadi hal itu tidak tergantung pada kehendak or...,18,16
2,START_ banjur diwartani pangandikane pangeran ...,lalu mereka memberitakan firman tuhan kepadany...,12,14
3,START_ nanging menawa aku nundhung setan ngang...,tetapi jika aku mengusir setan dengan kuasa al...,18,15
4,START_ pangagemane jubah kang wis kacelup ing ...,dan ia memakai jubah yang telah dicelup dalam ...,14,14


In [638]:
X, y = combined_data['text'], combined_data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((2196,), (550,))

In [639]:
X_train.head()

788     START_ sakabat loro iku mau tumuli padha mulih...
2040    START_ nanging saiki kowe ngegungake umukmu sa...
480     START_ sapa kang bakal nggugat para pilihane a...
900     START_ unjuke para sakabate lah sapunika panje...
2227    START_ wongwong kang padha krungu padha ngucap...
Name: text, dtype: object

In [640]:
y_train.head()

788               lalu pulanglah kedua murid itu ke rumah
2040    tetapi sekarang kamu memegahkan diri dalam con...
480     siapakah yang akan menggugat orangorang piliha...
900     kata muridmuridnya lihat sekarang engkau terus...
2227    dan mereka yang mendengar itu berkata jika dem...
Name: label, dtype: object

# Save data to PKL 

In [641]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')

In [642]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

# Encoder Decoder 

In [643]:
latent_dim=300

In [644]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)

encoder_states = [state_h, state_c]

In [645]:
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [646]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [647]:
model.summary()

Model: "functional_29"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_28      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_29      │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_28        │ (None, None, 300) │  2,027,400 │ input_layer_28[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_28        │ (None, None)      │          0 │ input_layer_28[0… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_29        │ (None, None, 300) │  2,872,800 │ input_layer_29[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_28 (LSTM)      │ [(None, 300),     │    721,200 │ embedding_28[0][… │
│                     │ (None, 300),      │            │ not_equal_28[0][… │
│                     │ (None, 300)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_29 (LSTM)      │ [(None, None,     │    721,200 │ embedding_29[0][… │
│                     │ 300), (None,      │            │ lstm_28[0][1],    │
│                     │ 300), (None,      │            │ lstm_28[0][2]     │
│                     │ 300)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_14 (Dense)    │ (None, None,      │  2,882,376 │ lstm_29[0][0]     │
│                     │ 9576)             │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 9,224,976 (35.19 MB)

 Trainable params: 9,224,976 (35.19 MB)

 Non-trainable params: 0 (0.00 B)

In [648]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 100

In [649]:
X_train.shape

(2196,)

In [650]:
X_test.shape

(550,)

In [651]:
y_train.shape

(2196,)

In [655]:
model.fit(
    [X_train, y_train], y_train,
    batch_size=128,
    epochs=100,
    validation_data=(X_test, y_test)
)

Epoch 1/100


ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(None, 1), output.shape=(None, 1, 9576)

In [652]:
model.fit(generate_batch(X_train, y_train, batch_size = batch_size),
          steps_per_epoch = train_samples//batch_size,
          epochs=epochs,
          validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
          validation_steps = val_samples//batch_size)

KeyError: 'START_'